# Tutorial no. 3 of SpeechBrain-MOABB: Integrating braindecode models

## **Prerequisites**


### Download SpeechBrain-MOABB

SpeechBrain-MOABB can be downloaded from the GitHub repository listed below.

In [ ]:
!git clone https://github.com/speechbrain/benchmarks

### Install SpeechBrain and SpeechBrain-MOABB requirements, and install SpeechBrain

In [ ]:
%%capture
!pip install speechbrain

%cd /content/benchmarks/benchmarks/MOABB
!pip install -r extra-requirements.txt # Install additional dependencies


### Install braindecode

In [ ]:
%%capture
!pip install braindecode

## **Define the yaml file integrating a braindecode model**

Based on the yaml file defined in the tutorial no. 0, let us assume that we are interested into using a model already developed in braindecode library. SpeechBrain-MOABB includes the model class `BraindecodeNN` that transforms a braindecode model for being compliant with SpeechBrain and SpeechBrain-MOABB. Then, the braindecode model can be integrated seamlessly with the usual decoding pipeline of SpeechBrain-MOABB. For example, for using EEGInception model, we can do the following.

In [ ]:
model_hyperparams = """
# MODEL
input_shape: [null, !ref <T>, !ref <C>, null]
braindecode_model: !new:braindecode.models.EEGInception
    n_outputs: !ref <n_classes>
    n_chans: !ref <C>
    n_times: !ref <T>
    add_log_softmax: True

model: !new:models.BraindecodeNN.BraindecodeNN
    model: !ref <braindecode_model>

"""

Then, we can add this model hyper-parameters to the other hyper-parameters defining the decoding pipeline.

In [ ]:
example_hyperparams = """
seed: 1234
__set_torchseed: !apply:torch.manual_seed [!ref <seed>]

# DIRECTORIES
data_folder: !PLACEHOLDER  #'/path/to/dataset'. The dataset will be automatically downloaded in this folder
cached_data_folder: !PLACEHOLDER #'path/to/pickled/dataset'
output_folder: !PLACEHOLDER #'path/to/results'

# DATASET HPARS
# Defining the MOABB dataset.
dataset: !new:moabb.datasets.BNCI2014001
save_prepared_dataset: True # set to True if you want to save the prepared dataset as a pkl file to load and use afterwards
data_iterator_name: 'leave-one-session-out'
target_subject_idx: 0
target_session_idx: 1
events_to_load: null # all events will be loaded
original_sample_rate: 250 # Original sampling rate provided by dataset authors
sample_rate: 125 # Target sampling rate (Hz)
# band-pass filtering cut-off frequencies
fmin: 1
fmax: 40
n_classes: 4
tmin: 0.
tmax: 4.0
# number of steps used when selecting adjacent channels from a seed channel (default at Cz)
n_steps_channel_selection: 3
T: !apply:math.ceil
    - !ref <sample_rate> * (<tmax> - <tmin>)
C: 22
test_with: 'best' # 'last' or 'best'
test_key: "acc" # Possible opts: "loss", "f1", "auc", "acc"

# METRICS
f1: !name:sklearn.metrics.f1_score
    average: 'macro'
acc: !name:sklearn.metrics.balanced_accuracy_score
cm: !name:sklearn.metrics.confusion_matrix
metrics:
    f1: !ref <f1>
    acc: !ref <acc>
    cm: !ref <cm>

# TRAINING HPARS
n_train_examples: 100  # it will be replaced in the train script
# checkpoints to average
avg_models: 1
number_of_epochs: 1000
lr: 0.001
# Learning rate scheduling (cyclic learning rate is used here)
max_lr: !ref <lr> # Upper bound of the cycle (max value of the lr)
base_lr: 0.00000001 # Lower bound in the cycle (min value of the lr)
step_size_multiplier: 5 #from 2 to 8
step_size: !apply:round
    - !ref <step_size_multiplier> * <n_train_examples> / <batch_size>
lr_annealing: !new:speechbrain.nnet.schedulers.CyclicLRScheduler
    base_lr: !ref <base_lr>
    max_lr: !ref <max_lr>
    step_size: !ref <step_size>
label_smoothing: 0.0
loss: !name:speechbrain.nnet.losses.nll_loss
    label_smoothing: !ref <label_smoothing>
optimizer: !name:torch.optim.Adam
    lr: !ref <lr>
epoch_counter: !new:speechbrain.utils.epoch_loop.EpochCounter  # epoch counter
    limit: !ref <number_of_epochs>
batch_size: 32
valid_ratio: 0.2

# DATA NORMALIZATION
dims_to_normalize: 1 # 1 (time) or 2 (EEG channels)
normalize: !name:speechbrain.processing.signal_processing.mean_std_norm
    dims: !ref <dims_to_normalize>

"""

example_hyperparams += model_hyperparams

In [ ]:
# Save the yaml file on disk
f = open('/content/example_hyperparams.yaml', "w")
f.write(example_hyperparams)
f.close()

## **Train the neural network on a single cross-validation fold**

Finally, we can train the network as done in tutorial no. 1. For brevity, we override also here the number of training epochs to 50 epochs.

In [ ]:
!python train.py /content/example_hyperparams.yaml \
--data_folder '/content/data/BNCI2014001' \
--cached_data_folder '/content/data' \
--output_folder '/content/results/single-fold-example-braindecode/BNCI2014001' \
--data_iterator_name 'leave-one-session-out' \
--target_subject_idx 0 \
--target_session_idx 1 \
--number_of_epochs 50